In [ ]:
from src.semseg_utils import *
from src.data import *
from models.SemSeg_Network import *
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

def tensor_to_image(tensor):
    tensor = tensor.detach().cpu().numpy().squeeze() 
    tensor_min, tensor_max = tensor.min(), tensor.max()
    if tensor_max > tensor_min:
        tensor = (tensor - tensor_min) / (tensor_max - tensor_min)
    return (tensor * 255).astype(np.uint8)  

data_path = '/root/home/MD/'

test_images = data_pred(data_path, 'images_ASHEVILLE_S_3Band50cm')
test_dataset = DataPrep(test_images, transform=transform)

BATCH_SIZE = 4

test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)


# for i in range(1):
i = 0
model = Network(1)

model_dict = torch.load('./pretrained_weights/DeepGlobe_SemSeg_SACloss.pth')

modified_model_dict0 = {k.replace('module.', ''): v for k, v in model_dict.items()}
modified_model_dict1 = {k.replace('segmentation.pcs_weight', 'segmentation.pcs_module.weight'): v for k, v in modified_model_dict0.items()}
modified_model_dict = {k.replace('segmentation.pcs_bias', 'segmentation.pcs_module.bias'): v for k, v in modified_model_dict1.items()}


model.load_state_dict(modified_model_dict)
model.to('cpu')  # or 'cuda' if you're using GPU
model.eval()
batch_idx = 0

for batch in test_loader:
    inputs = batch
    inputs = inputs.to('cpu')
    
    with torch.no_grad():
        pred1 = model(inputs)

    
    input_image = tensor_to_image(inputs[0])
    predicted_image = tensor_to_image(pred1[0])
    input_image = input_image.transpose(1, 2, 0) 

    # cv2.imwrite(f"/home/sara/Desktop/predictions/Road/SemSeg_ourLoss/models/SemSeg_CombinedLoss_ep{i}/predicted_cont0_{batch_idx}.png", predicted_cont[0])
    # cv2.imwrite(f"/home/sara/Desktop/predictions/Road/SemSeg_ourLoss/models/SemSeg_CombinedLoss_ep{i}/predicted_image0_{batch_idx}.png", predicted_image[0])
    # cv2.imwrite(f"/home/saraashojaeii/Desktop/Road Detection//predicted_{batch_idx}.png", predicted_image[1])
    
    # Visualization
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    ax[0].imshow(input_image, cmap='gray')
    ax[0].set_title('Input Image')
    ax[0].axis('off')
    ax[1].imshow(predicted_image[1], cmap='gray')
    ax[1].set_title('Predicted map 2')
    ax[1].axis('off')
    
    plt.show()




In [ ]:
from src.semseg_utils import *
from src.data import *
from models.SemSeg_Network import *
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

def tensor_to_image(tensor):
    tensor = tensor.detach().cpu().numpy().squeeze() 
    tensor_min, tensor_max = tensor.min(), tensor.max()
    if tensor_max > tensor_min:
        tensor = (tensor - tensor_min) / (tensor_max - tensor_min)
    return (tensor * 255).astype(np.uint8)  


data_path = '/root/home/MD/'
test_images, test_masks = data_pred(data_path, 'test', 'mass')

test_dataset = DataPrep(test_images, test_masks, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

i = 0
model = Network(1)

model_dict = torch.load('./pretrained_weights/DeepGlobe_SemSeg_SACloss.pth')
modified_model_dict0 = {k.replace('module.', ''): v for k, v in model_dict.items()}
modified_model_dict1 = {k.replace('segmentation.pcs_weight', 'segmentation.pcs_module.weight'): v for k, v in modified_model_dict0.items()}
modified_model_dict = {k.replace('segmentation.pcs_bias', 'segmentation.pcs_module.bias'): v for k, v in modified_model_dict1.items()}

model.load_state_dict(modified_model_dict)
model.to('cuda')  # or 'cuda' if you're using GPU
model.eval()
batch_idx = 0
    
test_count = 0
total_test_miou = 0
test_miou = 0    
test_class_iou = 0
total_test_class_iou = 0

test_comm = 0
test_corr = 0
test_qual = 0
total_test_comm = 0
total_test_corr = 0
total_test_qual = 0

for batch in tqdm(test_loader):
    test_x, test_y = batch
    test_x, test_y = test_x.to('cuda'), test_y.to('cuda') 
    
    with torch.no_grad():
        mask, x = model(test_x)

    x = x.detach()
    mask = mask.detach()
      
    mask = torch.argmax(mask, dim=1).detach().cpu().numpy()  
    test_y = test_y.squeeze().detach().cpu().numpy()  
    mask = mask.squeeze(0)

    comm, corr, qual = relaxed_f1(mask, test_y, 5)
    tmiou, ciou = mIoU(mask, test_y, 2)

    test_miou += tmiou
    test_class_iou += ciou

    test_comm += comm
    test_corr += corr
    test_qual += qual

    total_test_comm += test_comm
    total_test_corr += test_corr
    total_test_qual += test_qual

    total_test_miou += test_miou
    test_miou = 0
    test_comm = 0
    test_corr = 0
    test_qual = 0
    test_count += 1


total_test_class_iou = test_class_iou / test_count

test_comm_avg = total_test_comm / test_count
test_corr_avg = total_test_corr / test_count
test_qual_avg = total_test_qual / test_count


print(f"test_comm_avg: {test_comm_avg}, test_corr_avg: {test_corr_avg}, test_qual_avg: {test_qual_avg}")

In [ ]:
# Given values
recall_12 = test_comm_avg / 100
precision_12 = test_corr_avg / 100

# Calculate F1 score for the new values
f1_score_12 = 2 * (recall_12 * precision_12) / (recall_12 + precision_12)
f1_score_12

